In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/microsoft-malware-prediction/sample_submission.csv
/kaggle/input/microsoft-malware-prediction/train.csv
/kaggle/input/microsoft-malware-prediction/test.csv


# Importando as bibliotecas

In [2]:
import numpy as np
import pandas as pd 
import os
print(os.listdir("../input/microsoft-malware-prediction"))
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import time
train_sample_fraction = None
train_sample_num = 1500000
na_rate_threshold = 0.9
unbalanced_feature_rate_threshold = 0.9

['sample_submission.csv', 'train.csv', 'test.csv']


In [3]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
%%time
train = pd.read_csv('../input/microsoft-malware-prediction/train.csv', dtype=dtypes)

CPU times: user 2min 23s, sys: 9.57 s, total: 2min 33s
Wall time: 3min 17s


In [5]:
good_cols = list(train.columns)

for col in train.columns:
    
    # remove columns with high NA rate
    na_rate = train[col].isnull().sum() / train.shape[0]
    
    # remove columns with high Unbalanced values rate
    unbalanced_rate = train[col].value_counts(normalize=True, dropna=False).values[0]
    
    if na_rate > na_rate_threshold:
        good_cols.remove(col)
    elif unbalanced_rate > unbalanced_feature_rate_threshold:
        good_cols.remove(col)

In [6]:
good_cols

['MachineIdentifier',
 'EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'AVProductStatesIdentifier',
 'AVProductsInstalled',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'OsBuild',
 'OsSuite',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'IeVerIdentifier',
 'SmartScreen',
 'Census_MDC2FormFactor',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorManufacturerIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_PrimaryDiskTotalCapacity',
 'Census_PrimaryDiskTypeName',
 'Census_SystemVolumeTotalCapacity',
 'Census_TotalPhysicalRAM',
 'Census_ChassisTypeName',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_PowerPlatformRoleName',
 'Census_InternalBatteryType',
 'Census_InternalBatteryNumberOfCharges',
 'Census_OSVersion

In [7]:
train = train[good_cols]

In [8]:
import gc

gc.collect()

63

In [9]:
categorical_columns = list(train.loc[:, train.dtypes =="category"].columns)
numerical_and_binary_columns = list(train.loc[:, train.dtypes !="category"].columns)
numerical_columns = numerical_and_binary_columns

categorical_columns.remove("MachineIdentifier")

binary_columns = []
for col in (numerical_and_binary_columns):
    if train[col].nunique() == 2:
        binary_columns.append(col)
        numerical_columns.remove(col)

# Modelagem

In [10]:
if train_sample_fraction is not None:
    train_sample = train.sample(frac=train_sample_fraction, random_state=42)
elif train_sample_num is not None:
    train_sample = train.sample(n=train_sample_num, random_state=42)
else:
    train_sample = train.sample(n=1500000, random_state=42)

del train
gc.collect()

21

In [11]:
train_sample.shape

(1500000, 57)

In [12]:
test_dtypes = {k: v for k, v in dtypes.items() if k in good_cols}
test = pd.read_csv('../input/microsoft-malware-prediction/test.csv', dtype=test_dtypes, usecols=good_cols[:-1])

In [13]:
test.head()

,MachineIdentifier,EngineVersion,AppVersion,AvSigVersion,AVProductStatesIdentifier,AVProductsInstalled,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,...,Census_ActivationChannel,Census_IsFlightingInternal,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsTouchEnabled,Wdft_IsGamer,Wdft_RegionIdentifier
0,0000010489e3af074adeac69c53e555e,1.1.15400.5,4.18.1810.5,1.281.501.0,53447.0,1.0,43,58552.0,18.0,53.0,...,OEM:DM,NaN,NaN,807.0,8554.0,1,NaN,0,0.0,7.0
1,00000176ac758d54827acd545b6315a5,1.1.15400.4,4.18.1809.2,1.279.301.0,53447.0,1.0,68,71395.0,NaN,276.0,...,Retail,NaN,NaN,554.0,33105.0,1,NaN,0,1.0,12.0
2,0000019dcefc128c2d4387c1273dae1d,1.1.15300.6,4.18.1809.2,1.277.230.0,49480.0,2.0,201,66202.0,NaN,267.0,...,OEM:DM,NaN,NaN,556.0,63396.0,1,NaN,0,1.0,11.0
3,0000055553dc51b1295785415f1a224d,1.1.15400.5,4.18.1810.5,1.281.664.0,42160.0,2.0,29,120917.0,NaN,35.0,...,OEM:DM,NaN,0.0,628.0,26320.0,1,0.0,0,0.0,10.0
4,00000574cefffeca83ec8adf9285b2bf,1.1.15400.4,4.18.1809.2,1.279.236.0,53447.0,1.0,171,124736.0,18.0,211.0,...,Retail,NaN,NaN,556.0,63269.0,1,NaN,0,1.0,3.0


In [14]:
test.shape

(7853253, 56)

In [15]:
train_sample = train_sample.drop(['MachineIdentifier'], axis=1)
test = test.drop(['MachineIdentifier'], axis=1)

In [16]:
train_sample = train_sample.reset_index(drop=True)

# Preenchendo valores NA com o modo estatístico

In [17]:
modes = train_sample.mode()
for col in train_sample.columns:
    train_sample[col] = np.where(train_sample[col].isnull(), modes[col], train_sample[col])
del modes

In [18]:
modes_test = test.mode()
for col in test.columns:
    test[col] = np.where(test[col].isnull(), modes_test[col], test[col])
del modes_test

# Concatenar os conjuntos train_sample e test antes da codificação de rótulo

In [19]:
train_shape = train_sample.shape
test_shape = test.shape
train_and_test = pd.concat([train_sample,test], axis="rows", sort=False)
del train_sample
del test
gc.collect()

0

In [20]:
train_and_test.head()

,EngineVersion,AppVersion,AvSigVersion,AVProductStatesIdentifier,AVProductsInstalled,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,...,Census_IsFlightingInternal,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsTouchEnabled,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,1.1.15100.1,4.18.1807.18075,1.273.1234.0,53447.0,1.0,8,85219.0,27.0,205.0,-84,...,0.0,0.0,355.0,19951.0,0,0.0,0,0.0,11.0,1.0
1,1.1.15100.1,4.18.1807.18075,1.273.1282.0,53447.0,1.0,129,54198.0,27.0,126.0,124,...,0.0,0.0,486.0,48753.0,0,0.0,0,1.0,3.0,0.0
2,1.1.15100.1,4.9.10586.1106,1.273.781.0,46781.0,2.0,149,122689.0,27.0,181.0,56,...,0.0,0.0,554.0,33111.0,1,0.0,1,0.0,15.0,0.0
3,1.1.15200.1,4.18.1807.18075,1.275.1639.0,53447.0,1.0,164,114587.0,27.0,205.0,-84,...,0.0,0.0,142.0,48473.0,1,0.0,0,0.0,15.0,1.0
4,1.1.15200.1,4.18.1807.18075,1.275.511.0,47238.0,2.0,207,72421.0,27.0,277.0,75,...,0.0,0.0,355.0,4343.0,1,0.0,1,1.0,13.0,1.0


In [21]:
train_and_test.tail()

,EngineVersion,AppVersion,AvSigVersion,AVProductStatesIdentifier,AVProductsInstalled,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,...,Census_IsFlightingInternal,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsTouchEnabled,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
7853248,1.1.15400.5,4.18.1810.5,1.281.261.0,53447.0,1.0,29,157045.0,27.0,35.0,-85,...,0.0,0.0,554.0,33060.0,0,0.0,0,0.0,10.0,NaN
7853249,1.1.15400.5,4.18.1810.5,1.281.382.0,53447.0,1.0,95,145168.0,18.0,121.0,75,...,0.0,0.0,142.0,42119.0,0,0.0,0,0.0,11.0,NaN
7853250,1.1.15400.5,4.18.1810.5,1.281.261.0,68585.0,2.0,43,163420.0,18.0,53.0,42,...,0.0,0.0,152.0,8391.0,1,0.0,0,1.0,7.0,NaN
7853251,1.1.15400.4,4.8.10240.17443,1.279.1433.0,53447.0,1.0,159,56722.0,18.0,277.0,75,...,0.0,0.0,897.0,67854.0,0,0.0,0,0.0,11.0,NaN
7853252,1.1.15400.5,4.18.1810.5,1.281.796.0,53447.0,1.0,29,68856.0,27.0,35.0,-85,...,0.0,0.0,142.0,4966.0,1,0.0,0,0.0,10.0,NaN


# Codifique os recursos Categóricos antes da modelagem de aprendizado de máquina

In [22]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def MultiLabelEncoder(columnlist,dataframe):
    for i in columnlist:
        #print(i)
        labelencoder_X=LabelEncoder()
        dataframe[i]=labelencoder_X.fit_transform(dataframe[i])

MultiLabelEncoder(categorical_columns, train_and_test)

In [23]:
gc.collect()

21

# Voltar para treinar e testar o conjunto após a codificação de rótulo

In [24]:
train_sample = train_and_test[0:train_shape[0]]
test = train_and_test[(train_shape[0]):(train_and_test.shape[0]+1)]
test

,EngineVersion,AppVersion,AvSigVersion,AVProductStatesIdentifier,AVProductsInstalled,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,...,Census_IsFlightingInternal,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsTouchEnabled,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,68,70,9330,53447.0,1.0,43,58552.0,18.0,53.0,42,...,0.0,0.0,807.0,8554.0,1,0.0,0,0.0,7.0,NaN
1,67,64,9074,53447.0,1.0,68,71395.0,27.0,276.0,74,...,0.0,0.0,554.0,33105.0,1,0.0,0,1.0,12.0,NaN
2,65,64,8570,49480.0,2.0,201,66202.0,27.0,267.0,-5,...,0.0,0.0,556.0,63396.0,1,0.0,0,1.0,11.0,NaN
3,68,70,9370,42160.0,2.0,29,120917.0,27.0,35.0,-85,...,0.0,0.0,628.0,26320.0,1,0.0,0,0.0,10.0,NaN
4,67,64,9058,53447.0,1.0,171,124736.0,18.0,211.0,-74,...,0.0,0.0,556.0,63269.0,1,0.0,0,1.0,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7853248,68,70,9274,53447.0,1.0,29,157045.0,27.0,35.0,-85,...,0.0,0.0,554.0,33060.0,0,0.0,0,0.0,10.0,NaN
7853249,68,70,9295,53447.0,1.0,95,145168.0,18.0,121.0,75,...,0.0,0.0,142.0,42119.0,0,0.0,0,0.0,11.0,NaN
7853250,68,70,9274,68585.0,2.0,43,163420.0,18.0,53.0,42,...,0.0,0.0,152.0,8391.0,1,0.0,0,1.0,7.0,NaN
7853251,67,89,8950,53447.0,1.0,159,56722.0,18.0,277.0,75,...,0.0,0.0,897.0,67854.0,0,0.0,0,0.0,11.0,NaN


In [25]:
del train_and_test

# Removendo as colunas do HasDetections do conjunto de teste, elas foram adicionadas durante a concatenação do dataframe

In [26]:
test = test.drop(["HasDetections"], axis = 1)

In [27]:
y = train_sample['HasDetections']
X = train_sample.drop(['HasDetections'], axis=1)

In [28]:
del train_sample
gc.collect()

84

# XGBoost

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
import time

# create a 70/30 split of the data 
xtrain, xvalid, ytrain, yvalid = train_test_split(X, y, random_state=42, test_size=0.3)

import xgboost as xgb

start_time = time.time()

clf_xgb = xgb.XGBClassifier(learning_rate=0.1, 
                            n_estimators=1000, 
                            max_depth=5,
                            min_child_weight=1,
                            gamma=0,
                            subsample=0.9,
                            colsample_bytree=0.6,
                            objective= 'binary:logistic',
                            nthread=-1,
                            scale_pos_weight=1,
                            reg_alpha = 0,
                            reg_lambda = 1,
                            seed=42)

clf_xgb.fit(xtrain, ytrain, eval_set=[(xtrain, ytrain), (xvalid, yvalid)], 
            early_stopping_rounds=100, eval_metric='auc', verbose=100)

predictions = clf_xgb.predict(xvalid)

print(classification_report(yvalid, predictions))
print("accuracy_score", accuracy_score(yvalid, predictions))
predictions_probas = clf_xgb.predict_proba(xvalid)
print("roc-auc score for the class 1, from target 'HasDetections' ", roc_auc_score(yvalid, predictions_probas[:,1]))
print("elapsed time in seconds: ", time.time() - start_time)
gc.collect()

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  UserWarning,
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[0]	validation_0-auc:0.62197	validation_1-auc:0.62370
[100]	validation_0-auc:0.70808	validation_1-auc:0.70601
[200]	validation_0-auc:0.71661	validation_1-auc:0.71248
[300]	validation_0-auc:0.72196	validation_1-auc:0.71579
[400]	validation_0-auc:0.72572	validation_1-auc:0.71762
[500]	validation_0-auc:0.72924	validation_1-auc:0.71924
[600]	validation_0-auc:0.73221	validation_1-auc:0.72047
[700]	validation_0-auc:0.73487	validation_1-auc:0.72138
[800]	validation_0-auc:0.73722	validation_1-auc:0.72198
[900]	validation_0-auc:0.73951	validation_1-auc:0.72262
[999]	validation_0-auc:0.74152	validation_1-auc:0.72304
              precision    recall  f1-score   support

         0.0       0.66      0.66      0.66    225850
         1.0       0.66      0.65      0.66    224150

    accuracy                           0.66    450000
   macro avg       0.66      0.66      0.66    450000
weighted avg       0.66      0.66      0.66    450000

accuracy_score 0.6591288888888889
roc-auc score for the cla

30

# LGBM

# Gráficos